In [4]:
import os
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, storage
import uuid

In [2]:
data_cred={"type": "service_account",
		"project_id": os.getenv("project_id"),
		"private_key_id": os.getenv("private_key_id"),
		"private_key": os.getenv("private_key").replace('\\n', '\n'),
		"client_email": os.getenv("client_email"),
		"client_id": os.getenv("client_id"),
		"auth_uri": "https://accounts.google.com/o/oauth2/auth",
		"token_uri": "https://oauth2.googleapis.com/token",
		"auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
		"client_x509_cert_url": os.getenv("client_x509_cert_url"),
	"universe_domain":"googleapis.com"}

In [5]:
def initialize_firebase():
    # Check if any Firebase app exists
    if not firebase_admin._apps:
        # Initialize the default app
        cred = credentials.Certificate(data_cred)
        firebase_admin.initialize_app(cred)
    
    # Now, we can create a new app with a unique name if needed
    app = firebase_admin.initialize_app(credentials.Certificate(data_cred), name=str(uuid.uuid4()))
    
    return firestore.client(app)

 

In [6]:
# Use the function to get a Firestore client
db = initialize_firebase()

In [7]:
db

In [10]:
diagrambased_docs = []

# Get all documents from the collection
docs = db.collection("question-library").stream()

# Filter the documents in your application
for doc in docs:
    metadata = doc.get('metadata')
    if metadata and 'tags' in metadata:
        tags = metadata['tags']
        if len(tags) > 4 and tags[4] == 'diagram-based-question':
            diagrambased_docs.append(doc.id)

print(diagrambased_docs)

['QT2PdKq4MRP0PjQy47j4', 'foPFvZG2D5T0P9XSXupz']


In [17]:
docs = db.collection("question-library").stream()

# Filter and update the documents
for doc in docs:
    metadata = doc.get('metadata')
    if metadata and 'tags' in metadata:
        tags = metadata['tags']
        if len(tags) > 4 and tags[4] == 'activity-questions' and tags[3]=='MCQ':
            # Update the 'type' field
            doc_ref = db.collection("question-library").document(doc.id)
            doc_ref.update({"type": "multi-choice"})
            print(f"Updated document {doc.id}")

print("Update process completed")

Updated document 5rlcf59o8pEKvXadZfUE
Updated document Z91f38AmmYZmim5mSuZ1
Updated document nRwNq8hvQRVrTMzR8oYK
Updated document xfEKuSy3yfJv2Og2brhs
Updated document xrUBJ5Ij6hVCyNkkHHb9
Update process completed


In [ ]:
subject_option_ids = [doc.id for doc in db.collection("question-library").where("metadata.tags[4]", "==", "diagram-based-question").stream()]